#Installation of Required Libraries

In [ ]:
!pip install transformers==4.33.0 accelerate==0.22.0 einops==0.6.1 langchain==0.0.300 xformers==0.0.21 \
bitsandbytes==0.41.1 sentence_transformers==2.2.2 chromadb==0.4.12

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.6/151.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time
#import chromadb
#from chromadb.config import Settings
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

#Hugging Face CLI Authentication

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your termin

#Configure Model and Quantization Settings

In [ ]:
model_id = 'meta-llama/Llama-2-7b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

In [ ]:
time_1 = time()
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
time_2 = time()
print(f"Prepare model, tokenizer: {round(time_2-time_1, 3)} sec.")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Prepare model, tokenizer: 188.614 sec.


#Initialize and Time Text Generation Pipeline

In [ ]:
time_1 = time()
query_pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        device_map="auto",)
time_2 = time()
print(f"Prepare pipeline: {round(time_2-time_1, 3)} sec.")

Prepare pipeline: 1.536 sec.


**We define a function for testing the pipeline.**

In [ ]:
def test_model(tokenizer, pipeline, prompt_to_test):
    """
    Perform a query
    print the result
    Args:
        tokenizer: the tokenizer
        pipeline: the pipeline
        prompt_to_test: the prompt
    Returns
        None
    """
    # adapted from https://huggingface.co/blog/llama2#using-transformers
    time_1 = time()
    sequences = pipeline(
        prompt_to_test,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=200,)
    time_2 = time()
    print(f"Test inference: {round(time_2-time_1, 3)} sec.")
    for seq in sequences:
        print(f"Result: {seq['generated_text']}")

#Test the query pipeline

In [ ]:
test_model(tokenizer,
           query_pipeline,
           "Write a short note on Bayesian statistics. Keep it under 100 words.")

Test inference: 8.092 sec.
Result: Write a short note on Bayesian statistics. Keep it under 100 words.
Bayesian statistics is a statistical approach that uses Bayes' theorem to update probabilities based on new data or information. It differs from classical statistics in that it incorporates prior knowledge or beliefs about the parameters being estimated, and updates these probabilities based on the observed data. Bayesian statistics provides a framework for making probabilistic predictions and estimating parameters in a wide range of fields, including machine learning, engineering, and social sciences.


#Retrieval Augmented Generation

#Checking the model with a HuggingFace pipeline

In [ ]:
llm = HuggingFacePipeline(pipeline=query_pipeline)
# checking again that everything is working fine
llm(prompt="Write a short note on Bayesian statistics. Keep it under 100 words.")

"\n\nBayesian statistics is a statistical approach that uses Bayes' theorem to update probabilities based on new data or information. It differs from classical statistics in that it incorporates prior knowledge or beliefs about the parameter of interest, and updates these probabilities based on the observed data. Bayesian statistics is useful for estimating the probability of rare events or making predictions under uncertainty, and has applications in machine learning, engineering, and medical research."

#Ingestion of data using Text loder

In [ ]:
loader = TextLoader("/content/data/chapters1-19.txt",
                    encoding="utf8")
documents = loader.load()

#Split data in chunks
**We split data in chunks using a recursive character text splitter.**

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

#Set Up Sentence Transformers for Embeddings

In [ ]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}

embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

In [ ]:
vectordb = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory="chroma_db")

#Initialize chain

In [ ]:
retriever = vectordb.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True
)

#Test the Retrieval-Augmented Generation

In [ ]:
def test_rag(qa, query):
    print(f"Query: {query}\n")
    time_1 = time()
    result = qa.run(query)
    time_2 = time()
    print(f"Inference time: {round(time_2-time_1, 3)} sec.")
    print("\nResult: ", result)

**Let's check few queries.**

In [ ]:
query = "Write a short note on Bayesian statistics. Keep it under 200 words."
test_rag(qa, query)

Query: Write a short note on Bayesian statistics. Keep it under 200 words.



> Entering new RetrievalQA chain...

> Finished chain.
Inference time: 16.455 sec.

Result:   Bayesian statistics is a method of statistical inference that uses Bayes' theorem to update probabilities based on new data or information. It involves making an educated guess about the causes of a phenomenon given the known effects. The goal is to deduce a probability function on the set of possible causes, granted that we have the probabilities of the various effects. Bayesian statistics is useful when there are multiple possible causes for a phenomenon, and it can be used to incorporate prior knowledge or beliefs about the causes into the analysis. However, it is important to note that Bayesian statistics is not always the best approach, and it may not be appropriate in situations where the data is noisy or limited.


In [ ]:
query = "What are Common probability functions. Keep it under 200 words."
test_rag(qa, query)

Query: What are Common probability functions. Keep it under 200 words.



> Entering new RetrievalQA chain...

> Finished chain.
Inference time: 15.834 sec.

Result:   Common probability functions are mathematical models used to determine if a given phenomenon is unexpected. These functions include the normal distribution, the Poisson distribution, and the binomial distribution. The normal distribution is used to model continuous random variables, while the Poisson distribution is used for count data and the binomial distribution for discrete random variables. Understanding these probability functions is essential in Bayesian statistics, as they provide a framework for evaluating the likelihood of a particular outcome.


#Query Execution and Document Retrieval

In [ ]:
docs = vectordb.similarity_search(query)
print(f"Query: {query}")
print(f"Retrieved documents: {len(docs)}")
for doc in docs:
    doc_details = doc.to_json()['kwargs']
    print("Source: ", doc_details['metadata']['source'])
    print("Text: ", doc_details['page_content'], "\n")

Query: What are Common probability functions. Keep it under 200 words.
Retrieved documents: 4
Source:  /content/data/chapters1-19.txt
Text:  13.3. The mean and standard deviation                                                                                     81
13.5 Examples of probability functions
Three examples of continuous probability functions appear regularly in the scientific literature. 

Source:  /content/data/chapters1-19.txt
Text:  58                                                                                       Chapter 10. More about Bayesian statistics
                                                                                                             CHAPTER

                                                                                                           ELEVEN



                                    Common probability functions

There are certain probability functions that serve as models that are commonly used when trying to decide if a gi